# Ingest pit_stops.json file

## Pre-requisites
- Add data source and file date parameter widgets to the notebook and get it as a variable
- Run configuration notebook with folder paths
- Run common_functions notebook

In [0]:
dbutils.widgets.text("param_data_source", "")
var_data_source = dbutils.widgets.get("param_data_source")

In [0]:
dbutils.widgets.text("param_file_date", "")
var_file_date = dbutils.widgets.get("param_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

### Add ingestion date column to each data frame using current timestamp

### Step 1 - Read the JSON file using the Spark DataFrame Reader

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

In [0]:
pit_stops_schema = StructType(fields = [
    StructField("raceId", IntegerType(), False),
    StructField("driverId", IntegerType(), True),
    StructField("stop", IntegerType(), True),
    StructField("lap", IntegerType(), True),
    StructField("time", StringType(), True),
    StructField("duration", StringType(), True),
    StructField("milliseconds", IntegerType(), True)
])

In [0]:
pit_stops_df = spark.read \
    .schema(pit_stops_schema) \
    .option("multiLine", True) \
    .json(f'{raw_folder_path}/{var_file_date}/pit_stops.json')

## Step 2 - Rename columns and add ingestion date column

In [0]:
from pyspark.sql.functions import lit

In [0]:
pit_stops_renamed_df = pit_stops_df.withColumnRenamed("driverId", "driver_id") \
                                    .withColumnRenamed("raceId", "race_id") \
                                    .withColumn("data_source", lit(var_data_source)) \
                                    .withColumn("file_date", lit(var_file_date))

In [0]:
pit_stops_final_df = add_ingestion_date(pit_stops_renamed_df)

## Step 3 - Write data to Data Lake as a managed table in Delta Lake format

In [0]:
merge_condition = "tgt.race_id = src.race_id AND tgt.driver_id = src.driver_id AND tgt.stop = src.stop"
merge_delta_data(pit_stops_final_df, 'f1_processed', 'pit_stops', processed_folder_path, merge_condition, 'race_id')

In [0]:
dbutils.notebook.exit("Success")